## 데이터 분석 목적

고등학생을 대상으로 수학 성적과 참여한 프로그램의 유형에 따라 1년에 상을 받은 횟수가 어떻게 변하는지 확인

In [16]:
# https://zephyrus1111.tistory.com/88
import pandas as pd
import numpy as np 
import statsmodels.api as sm

from functools import reduce
from scipy.stats import norm

df = pd.read_csv('../rawdata/num_awards.csv')

In [17]:
df.head()

,id,num_awards,prog,math
0,45,0,3,41
1,108,0,1,41
2,15,0,3,44
3,67,0,3,42
4,153,0,3,40


In [18]:
df.describe()

,id,num_awards,prog,math
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,0.630000,2.025000,52.645000
std,57.879185,1.052921,0.690477,9.368448
min,1.000000,0.000000,1.000000,33.000000
25%,50.750000,0.000000,2.000000,45.000000
50%,100.500000,0.000000,2.000000,52.000000
75%,150.250000,1.000000,2.250000,59.000000
max,200.000000,6.000000,3.000000,75.000000


In [19]:
df['prog'].value_counts()

2    105
3     50
1     45
Name: prog, dtype: int64

In [20]:
df = df[['num_awards', 'prog', 'math']]
df = pd.get_dummies(df, columns=['prog'], drop_first=True)
df = sm.add_constant(df) # 절편향 추가

## 모형 적합

반복 횟수가 100을 넘거나 회귀 계수의 현재 값과 다음 값의 차이를 계산하고 이 차이값이 0.0001보다 작으면 반복 멈춤

In [21]:
df.head()

,const,num_awards,math,prog_2,prog_3
0,1.0,0,41,0,1
1,1.0,0,41,0,0
2,1.0,0,44,0,1
3,1.0,0,42,0,1
4,1.0,0,40,0,1


In [22]:
y = df['num_awards']
X = np.array(df[['const','math','prog_2','prog_3']])
 
## initial beta
y_mean = y.mean()*np.ones(len(y))
 
X_tX = np.matmul(X.T,X)
X_tX_inv = np.linalg.inv(X_tX)
beta = reduce(np.dot,[X_tX_inv, X.T, np.log(y_mean)])
# beta = [0]*X.shape[1] ## 베타의 초기값을 0으로 해도 좋다.
 
epsilon = 0.0001 ## tolerance
max_iter = 100
iter_count = 0
 
while iter_count <= max_iter:
    iter_count += 1
    eta = np.matmul(X,beta)
    mu = np.exp(eta)
    D = np.diag(mu)
    D_inv = np.linalg.inv(D)
        
    z = eta + np.matmul(D_inv,y-mu)
    
    W = np.diag(mu)
    X_tWX = np.matmul(np.matmul(X.transpose(),W),X)
    X_tWX_inv = np.linalg.inv(X_tWX)
    X_tWz = np.matmul(np.matmul(X.transpose(),W),z)
    next_beta = np.matmul(X_tWX_inv,X_tWz)
    diff = np.linalg.norm(next_beta-beta)
    
    beta = next_beta
    if diff < epsilon:
        break

In [23]:
beta

array([-5.2471244 ,  0.0701524 ,  1.08385914,  0.36980923])

In [24]:
## 신뢰구간
alpha = 0.05
columns = ['const','math','prog_2','prog_3']
for i in range(len(beta)):
    upper_limit = beta[i] + norm.ppf(1-alpha/2)*np.sqrt(X_tWX_inv[i][i])
    lower_limit = beta[i] - norm.ppf(1-alpha/2)*np.sqrt(X_tWX_inv[i][i]) 
    print(columns[i], lower_limit, upper_limit)

const -6.537652484192229 -3.956596308795824
math 0.04937845343909553 0.09092634155762247
prog_2 0.3817169890833214 1.7860012975081947
prog_3 -0.49464732173558734 1.2342657790363154


## Possion Log Linear 모형 적합

In [25]:
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families import Poisson

In [26]:
y = df['num_awards']
X = df[['const','math','prog_2','prog_3']]
 
model = GLM(y,X,family=Poisson())
results = model.fit()

In [27]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             num_awards   No. Observations:                  200
Model:                            GLM   Df Residuals:                      196
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -182.75
Date:                Mon, 23 Aug 2021   Deviance:                       189.45
Time:                        21:00:56   Pearson chi2:                     212.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.2471      0.658     -7.969      0.000      -6.538      -3.957
math           0.0702      0.011      6.619      0.000       0.049       0.091
prog_2         1.0839      0.358      3.025      0.002       0.382       1.786
prog_3         0.3698      0.441      0.838      0.402      -0.495       1.234
==============================================================================
"""